In [23]:
import cv2
import numpy as np
import time
from Person import Person

In [2]:
# check opencv is working
print('Currently running OpenCV', cv2.__version__)

Currently running OpenCV 3.4.3


In [3]:
file_list = ['../Videos/Hallway_002.avi','../Videos/Hallway_003.avi','../Videos/Shop_Corridor.mp4','../Videos/Shop_Front.mp4','../Videos/Student_Services.mp4']
title_list = ['Hallway_002','Hallway_003','Shop_Corridor','Shop_Front','Student_Services']

In [21]:
# open results file
results = open('BS Results.txt', "w+")

# Hallway_002
---

In [24]:
file = file_list[0]
title = title_list[0]

# open capture
cap = cv2.VideoCapture(file)

# get length of sequence
cap_length = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
cap_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
cap_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

# initialize detector
detector = cv2.createBackgroundSubtractorMOG2(history=100, varThreshold=40)

# people detections
people = []

start = time.time()
for _ in range(cap_length):
    
    # read frame
    read_ok, frame = cap.read()
    
    # apply detector
    gray = detector.apply(frame)
    _, bw = cv2.threshold(gray, 254, 255, cv2.THRESH_BINARY)
    
    # find contours
    _, contours, _ = cv2.findContours(bw, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
    
    # turn large contours into a list of bounding boxes
    detected_boxes = []
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 120:
            x, y, w, h = cv2.boundingRect(cnt)
            detected_boxes.append([x, y, w, h])
    
    # turn boxes into centroids
    detected_centroids = []
    for box in detected_boxes:
        x, y, w, h = box
        centroid = ((x+x+w)//2, (y+y+h)//2)
        detected_centroids.append(centroid)
    
    # loop through people and update
    for person in people:
        for centroid in detected_centroids:
            if person.compare(centroid):
                person.update(centroid)
            else:
                person.update(None)
    
    # loop through centroids and add new people
    for centroid in detected_centroids:
        match = False
        for person in people:
            if person.compare(centroid):
                match = True
        if match==False:
            people.append(Person(centroid, 20))
    
    # Prune expired people detections
    for person in people:
        if not person.isValid():
            people.remove(person)
    
    # convert back to 3-channel
    #gray = cv2.cvtColor(gray,cv2.COLOR_GRAY2RGB)
    #bw = cv2.cvtColor(bw,cv2.COLOR_GRAY2RGB)
    
    # Draw on frame
    #cv2.putText(bw, 'FPS: ' + str(fps), (50,50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1, cv2.LINE_AA)
    #cv2.putText(bw, 'People: ' + str(len(people)), (50,150), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 1, cv2.LINE_AA)
    #for person in people:
        #for point in person.path:
            #cv2.circle(bw,point, 2, (0,255,0), -1)
    
end = time.time()

acc_time = end-start
fps = cap_length/acc_time

results.write(title + '\n')
results.write('==================================================\n')
results.write('Frame Size: ' + str(cap_width) + 'x' + str(cap_height) + '\n')
results.write('Number of Frames: ' + str(cap_length) + '\n')
results.write('Sequence executed in: ' + str(round(acc_time,2)) + ' seconds\n')
results.write('FPS: '+ str(round(fps,2)) + '\n')
results.write('\n')

KeyboardInterrupt: 

In [15]:
results.close()